#Importação de Bibliotecas

In [ ]:
!pip install inflection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import inflection
import plotly.express as px
import folium
import webcolors
from folium.plugins import MarkerCluster
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/Othercomputers/Note Avell/repos/FTC/Ciclo 07 Projeto Final/'
df = pd.read_csv(path + 'zomato.csv')

# Funções

In [ ]:
#Nome dos países com base no código
countries = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}
#Função para substituir o código dos países pelo nome dos países no Dataframe
def country_name(country_id):
  return countries[country_id]

#Função para substituir o número do price_range pelo nome dos price_range no Dataframe
def create_price_type(price_range):
  if price_range == 1:
    return "cheap"
  elif price_range == 2:
    return "normal"
  elif price_range == 3:
    return "expensive"
  else:
    return "gourmet" 

#Código das Cores
colors = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}

#Função para substituir o código das cores pelo nome das cores no Dataframe
def color_name(color_code):
  return colors[color_code]

#Código das Moedas
codigos_moeda = {
 'Botswana Pula(P)': 'BWP',   
 'Brazilian Real(R$)': 'BRL',
 'Dollar($)': 'USD',
 'Emirati Diram(AED)': 'AED',
 'Indian Rupees(Rs.)': 'INR',
 'Indonesian Rupiah(IDR)': 'IDR',
 'NewZealand($)': 'NZD',
 'Pounds(£)': 'GBP',
 'Qatari Rial(QR)': 'QAR',
 'Rand(R)': 'ZAR',
 'Sri Lankan Rupee(LKR)': 'LKR',
 'Turkish Lira(TL)': 'TRY', 
}

#Função para substituir o nome das moedas pelo código das moedas no Dataframe
def renomear_moedas(moeda):
  return codigos_moeda[moeda]

#Taxa de conversão das moedas para dólar
taxa_dolar =  {
 'BWP': 13.0675,
 'BRL': 4.8828,
 'USD': 1.0000,
 'AED': 3.6607, 
 'INR': 81.6273, 
 'IDR': 14817.5759, 
 'NZD': 1.5897, 
 'GBP': 0.8010, 
 'QAR': 3.64,
 'ZAR': 18.0567, 
 'LKR': 305.9658, 
 'TRY': 19.1605,  
}

#Função para substituir o valor da coluna average_cost_for_two pelo valor convertido em dólar
"""
Função Tranforma a Moeda desejada em Dólar

Parâmetros: Valor -> valor que deseja transformar em dólar(float)
            moeda -> código da moeda de três digitos(string)
"""
def moeda_dolar(valor,moeda):
  return valor/taxa_dolar[moeda]

#Função para renomear as colunas
#Deixar todos os nomes com letra minúscula e substituir os espaços por "_"
def rename_columns(dataframe):
  df = dataframe.copy()
  title = lambda x: inflection.titleize(x)
  snakecase = lambda x: inflection.underscore(x)
  spaces = lambda x: x.replace(" ", "")
  cols_old = list(df.columns)
  cols_old = list(map(title, cols_old))
  cols_old = list(map(spaces, cols_old))
  cols_new = list(map(snakecase, cols_old))
  df.columns = cols_new
  return df

#Função para retirar Outliers da coluna average_cost_for_two baseado na mediana
def mediana_pais(df,pais):
  df2 = df.copy()
  filtro_pais = df2['country_code'] == pais
  mediana = df2.loc[filtro_pais,['average_cost_for_two']].median().values[0]
  filtro_mediana = df2['average_cost_for_two'] > 20*mediana
  df2.loc[(filtro_pais) & (filtro_mediana),'average_cost_for_two'] = mediana
  return df2   

#Função que encontra restaurante com maior ou menor nota baseado na culinária selecionada
"""
Função Calcula o melhor ou o pior nome de restaurante baseado na nota média e baseado no tipo de culinário

Parâmetros: cuisine -> tipo de culinária(string)
            min_max -> True para pior restaurante, False para melhor restaurante(boolean)
"""
def restaurant_by_cuisine(cuisine,min_max):
  filtro_cuisine_american = df2['cuisines'] == cuisine
  df2_filtrado = (df2.loc[filtro_cuisine_american,['restaurant_name','aggregate_rating','restaurant_id']]
              .groupby(['restaurant_name','restaurant_id'])
              .mean()
              .reset_index()
              .sort_values(by='aggregate_rating',ascending=min_max))
  df2_apenas_melhores_medias = df2_filtrado['aggregate_rating'] == df2_filtrado.iloc[0,2]
  df2_desempate = df2_filtrado.loc[df2_apenas_melhores_medias,:].sort_values(by='restaurant_id',ascending=True)
  if min_max:
    return print(f"O restaurante com a menor nota média que possui apenas o tipo de culinária {cuisine} é {df2_desempate.iloc[0,0]} com a média de {df2_desempate.iloc[0,2]}")
  else:
    return print(f"O restaurante com a maior nota média que possui apenas o tipo de culinária {cuisine} é {df2_desempate.iloc[0,0]} com a média de {df2_desempate.iloc[0,2]}")  

# Limpeza dos Dados

In [ ]:
df1 = df.copy()

#Retirando linhas duplicadas
df1 = df1.drop_duplicates()

#Retirando NaN
df1 = df1.dropna()

#Transformando Código dos países em países
df1['Country Code'] = df1['Country Code'].apply(country_name)

#Price Range das comidas
df1['Price range'] = df1['Price range'].apply(create_price_type)

#Mudar código de cores
df1['Rating color'] = df1['Rating color'].apply(color_name)

#Mudar código das moedas
df1['Currency'] = df1['Currency'].apply(renomear_moedas)

#Mudar nome colunas
df1 = df1.reset_index(drop=True)
df2 = df1.copy()
df2 = rename_columns(df1)

#Deixar apenas um nome na culinária
df2['cuisines'] = df2['cuisines'].astype(str)
df2["cuisines"] = df2.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

#Convertendo valores da coluna 'average_cost_for_two' em dólares americanos(USD)
df2['average_cost_for_two'] = df2.apply(lambda x: moeda_dolar(x['average_cost_for_two'],x['currency']),axis=1)
df2['average_cost_for_two'] = df2['average_cost_for_two'].round(decimals=2)

#Substituir Outliers de da coluna average_cost_for_two baseado na mediana
lista_paises = list(df2['country_code'].unique())
for pais in lista_paises:
  df2 = mediana_pais(df2,pais) 

#Mostrar todas as colunas na visualização
pd.set_option('display.max_columns', None) 

<ipython-input-7-8bf6c74bcf71>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Country Code'] = df1['Country Code'].apply(country_name)
<ipython-input-7-8bf6c74bcf71>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Price range'] = df1['Price range'].apply(create_price_type)
<ipython-input-7-8bf6c74bcf71>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

# Visão Geral

## 1. Quantos restaurantes únicos estão registrados?

In [ ]:
df2['average_cost_for_two'].min()

0.0

In [ ]:
rest_uniq = df2['restaurant_id'].nunique()
print(f"Existem {rest_uniq} restaurantes únicos")

Existem 6929 restaurantes únicos


## 2. Quantos países únicos estão registrados?

In [ ]:
country_uniq = df2['country_code'].nunique()
print(f"Existem {country_uniq} países únicos")

Existem 15 países únicos


## 3. Quantas cidades únicas estão registradas?

In [ ]:
city_uniq = df2['city'].nunique()
print(f"Existem {city_uniq} cidades únicas")

Existem 125 cidades únicas


## 4. Qual o total de avaliações feitas?

In [ ]:
print(f"O total de avaliações é {df2['votes'].sum()}")

O total de avaliações é 4194533


## 5. Qual o total de tipos de culinária registrados?

In [ ]:
cuisine_uniq = df2['cuisines'].nunique()
print(f"Existem {cuisine_uniq} tipos de culinária registrados")

Existem 165 tipos de culinária registrados


# Visão País

## 1. Qual o nome do país que possui mais cidades registradas?

In [ ]:
cols = ['city','country_code']
df2_aux = df2.loc[:,cols].groupby('country_code').nunique().reset_index()
df2_aux = df2_aux.sort_values(by='city',ascending=False)
print(f"O país que mais possui cidades registradas é {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,1]} cidades registradas")
px.bar(df2_aux,x='country_code',y='city',labels={'country_code: País', 'city: Cidades'})

O país que mais possui cidades registradas é India com 49 cidades registradas


## 2. Qual o nome do país que possui mais restaurantes registrados?

In [ ]:
cols = ['restaurant_id','country_code']
df2_aux = df2.loc[:,cols].groupby('country_code').nunique().reset_index()
df2_aux = df2_aux.sort_values(by='restaurant_id',ascending=False)
print(f"O país que mais possui restaurantes registrados é {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,1]} restaurantes registrados")
px.bar(df2_aux,x='country_code',y='restaurant_id',labels={'country_code: País', 'restaurant_id: Restaurantes'})

O país que mais possui restaurantes registrados é India com 3111 restaurantes registrados


## 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [ ]:
df2_aux = df2['price_range'] == 'gourmet'
df2_gourmet = df2.loc[df2_aux,:]
cols = ['country_code','restaurant_id']
df2_final = df2_gourmet.loc[:,cols].groupby('country_code').count().reset_index()
df2_final = df2_final.sort_values(by='restaurant_id',ascending=False)
print(f"O país com mais restaunte com nível de preço 4 é {df2_final.iloc[0,0]} com {df2_final.iloc[0,1]} restaurantes")

#Gráfico de barras exemplificando
px.bar(df2_final, x='country_code',y='restaurant_id', labels={'country_code': 'Países', 'restaurant_id': 'Qtd Restaurantes'})

O país com mais restaunte com nível de preço 4 é United States of America com 415 restaurantes


## 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [ ]:
cols = ['country_code','cuisines']
df2_aux = df2.loc[:,cols].groupby('country_code').nunique().reset_index()
df2_aux = df2_aux.sort_values(by='cuisines', ascending=False)
print(f"O País que mais possui quantidade de culinárias distintos é {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,1]} culinárias distintas")
px.bar(df2_aux,x='country_code',y='cuisines',labels={'country_code: País', 'cuisines: Culinárias'})

O País que mais possui quantidade de culinárias distintos é India com 77 culinárias distintas


## 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [ ]:
#agregar por city e aggregate_rating
cols = ['country_code','votes']
df2_aux = df2.loc[:,cols].groupby('country_code').sum().reset_index()
df2_aux = df2_aux.sort_values(by='votes', ascending=False)
print(f"O País que mais possui quantidade de avaliações feitas é {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,1]} avaliações feitas")
px.bar(df2_aux,x='country_code',y='votes',labels={'country_code: País', 'votes: Avaliações'})

O País que mais possui quantidade de avaliações feitas é India com 2800164 avaliações feitas


## 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [ ]:
#Saída -> pais: qtd restaurantes
cols = ['restaurant_id','country_code']
df2_delivery = df2['is_delivering_now'] == 1
df2_aux_delivery = df2.loc[df2_delivery,:]
df2_final = df2_aux_delivery.loc[:,cols].groupby('country_code').nunique().reset_index()
print(f"O país que possui a maior quantidade de restaurantes que fazem entrega é a {df2_final.iloc[0,0]} com {df2_final.iloc[0,1]} restaurantes")

O país que possui a maior quantidade de restaurantes que fazem entrega é a India com 1150 restaurantes


## 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?



In [ ]:
df2_only_reserv = df2.loc[df2['has_table_booking'] == 1,:]
cols = ['country_code','restaurant_id']
df2_aux = df2_only_reserv.loc[:,cols].groupby('country_code').nunique().reset_index()
df2_aux = df2_aux.sort_values(by='restaurant_id',ascending=False)
print(f"O País que possui a maior quantidade de restaurantes que aceitam reservas é {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,1]} restaurantes")

O País que possui a maior quantidade de restaurantes que aceitam reservas é India com 256 restaurantes


## 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?



In [ ]:
cols = ['country_code','votes']
df2_aux = df2.loc[:,cols].groupby('country_code').mean().reset_index().sort_values(by='votes',ascending=False)
print(f"O país que possui, na média, a maior quantidade de avaliações é {df2_aux.iloc[0,0]} com {np.round(df2_aux.iloc[0,1],decimals=0)} avaliações")

O país que possui, na média, a maior quantidade de avaliações é Indonesia com 1113.0 avaliações


## 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [ ]:
cols = ['aggregate_rating', 'country_code']
df2_aux = df2.loc[:,cols].groupby('country_code').agg({'aggregate_rating': ['mean','std','count']})
df2_aux.columns = ['media_rating','std_rating','count_rating']
df2_aux = df2_aux.sort_values(by='media_rating',ascending=False)
df2_aux = df2_aux.reset_index()
print(f"O País que possui a maior média de avalições registrada é {df2_aux.iloc[0,0]} com nota de {np.round(df2_aux.iloc[0,1],decimals=2)}")
px.bar(df2_aux,x='country_code',y='media_rating',error_y='std_rating',labels={'country_code': 'País', 'media_rating': 'Nota Média'})

O País que possui a maior média de avalições registrada é Indonesia com nota de 4.6


## 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [ ]:
cols = ['aggregate_rating', 'country_code']
df2_aux = df2.loc[:,cols].groupby('country_code').agg({'aggregate_rating': ['mean','std','count']})
df2_aux.columns = ['media_rating','std_rating','count_rating']
df2_aux = df2_aux.sort_values(by='media_rating',ascending=True)
df2_aux = df2_aux.reset_index()
print(f"O País que possui a menor média de avalições registrada é {df2_aux.iloc[0,0]} com nota de {np.round(df2_aux.iloc[0,1],decimals=2)}")

O País que possui a menor média de avalições registrada é Brazil com nota de 3.32


## 11. Qual a média de preço de um prato para dois por país?

In [ ]:
cols = ['average_cost_for_two','country_code',]
df2_aux = df2.loc[:,cols].groupby(['country_code']).mean().reset_index().sort_values(by='average_cost_for_two',ascending=False)
px.bar(df2_aux,x='country_code',y='average_cost_for_two',labels={'country_code': 'País', 'average_cost_for_two': 'Preço Médio'})

# Visão Cidade

## 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [ ]:
df2_aux = df2.loc[:,['city','restaurant_id','country_code']].groupby(['city','country_code']).nunique().reset_index().sort_values(by='restaurant_id',ascending=False)
print(f"A cidade que possui a maior quantidade de restaurantes é {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,2]} restaurantes")
px.bar(df2_aux.head(30),x='city',y='restaurant_id',color='country_code',labels={'city': 'Cidades', 'restaurant_id': 'Restaurantes','country_code':'Países'})

A cidade que possui a maior quantidade de restaurantes é Abu Dhabi com 80 restaurantes


## 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [ ]:
filtro_maior_4 = df2.loc[:,'aggregate_rating'] > 4
df2_aux = df2.loc[filtro_maior_4,['restaurant_id','city','country_code']].groupby(['city','country_code']).nunique().reset_index().sort_values(by='restaurant_id',ascending=False)
print(f"A cidade que possui mais restaurantes com média acima de 4 é {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,2]} restaurantes")
px.bar(df2_aux.head(30),x='city',y='restaurant_id',color='country_code',labels={'city': 'Cidades', 'restaurant_id': 'Restaurantes','country_code':'Países'})

A cidade que possui mais restaurantes com média acima de 4 é Bangalore com 79 restaurantes


## 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [ ]:
filtro_menor_25 = df2.loc[:,'aggregate_rating'] < 2.5
df2_aux = df2.loc[filtro_menor_25,['restaurant_id','city','country_code']].groupby(['city','country_code']).nunique().reset_index().sort_values(by='restaurant_id',ascending=False)
print(f"A cidade que possui mais restaurantes com média abaixo de 2.5 é {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,2]} restaurantes")
px.bar(df2_aux.head(30),x='city',y='restaurant_id',color='country_code',labels={'city': 'Cidades', 'restaurant_id': 'Restaurantes','country_code':'Países'})

A cidade que possui mais restaurantes com média abaixo de 2.5 é Gangtok com 33 restaurantes


## 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [ ]:
df2_aux = df2.loc[:,['city','average_cost_for_two','country_code']].groupby(['city','country_code']).mean().reset_index().sort_values(by='average_cost_for_two',ascending=False)
print(f"A cidade que possui o maior valor média de um prato para dois é {df2_aux.iloc[0,0]} com o valor de {df2_aux.iloc[0,2]}")
px.bar(df2_aux.head(30),x='city',y='average_cost_for_two',color='country_code',labels={'city': 'Cidades', 'average_cost_for_two': 'Preço Médio para 2','country_code':'Países'})

A cidade que possui o maior valor média de um prato para dois é Pasay City com o valor de 306.1


## 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [ ]:
df2_aux = df2.loc[:,['city','cuisines']].groupby('city').nunique().reset_index().sort_values(by='cuisines', ascending=False)
print(f"A cidade que possui a maior quantidade de tipos de culinária distintas é {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,1]} culinárias")

A cidade que possui a maior quantidade de tipos de culinária distintas é Birmingham com 32 culinárias


## 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [ ]:
filtro_apenas_reserva = df2['has_table_booking'] == 1
cols = ['city','restaurant_id']
df2_aux = df2.loc[filtro_apenas_reserva,cols].groupby('city').count().reset_index().sort_values(by='restaurant_id',ascending=False)
print(f"A cidade que possui a maior quantidade de restaurantes que fazem reservas é {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,1]} restaurantes")

A cidade que possui a maior quantidade de restaurantes que fazem reservas é Bangalore com 42 restaurantes


## 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [ ]:
cols = ['restaurant_id','city','country_code']
df2_delivery = (df2['is_delivering_now'] == 1)
df2_aux_delivery = df2.loc[df2_delivery,:]
df2_final = df2_aux_delivery.loc[:,cols].groupby(['city','country_code']).nunique().reset_index()
df2_final = df2_final.sort_values(by='restaurant_id',ascending=False)
print(f"A Cidade que possui a maior quantidade de restaurantes que fazem entrega é a {df2_final.iloc[0,0]} com {df2_final.iloc[0,2]} restaurantes")

px.bar(df2_final.head(30), x='city',y='restaurant_id',color='country_code', 
       labels={'city':'cidade',
               'restaurant_id': 'Restaurantes',
               'country_code': 'Países'},
               )

A Cidade que possui a maior quantidade de restaurantes que fazem entrega é a Vadodara com 48 restaurantes


## 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [ ]:
filtro_apenas_reserva = df2['has_online_delivery'] == 1
cols = ['city','restaurant_id','country_code']
df2_aux = df2.loc[filtro_apenas_reserva,cols].groupby(['city','country_code']).count().reset_index().sort_values(by='restaurant_id',ascending=False)
print(f"A cidade que possui a maior quantidade de restaurantes que aceitam pedidos online é {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,2]} restaurantes")
px.bar(df2_aux.head(30), x='city',y='restaurant_id',color='country_code', 
       labels={'city':'cidade',
               'restaurant_id': 'Restaurantes',
               'country_code': 'Países'},
               )

A cidade que possui a maior quantidade de restaurantes que aceitam pedidos online é Bhopal com 75 restaurantes


# Visão Restaurantes

## 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [ ]:
df2_aux = df2.loc[:,['restaurant_name','votes']].groupby('restaurant_name').sum().reset_index().sort_values(by='votes',ascending=False)
print(f"O restaurante que possui a maior quantidade de avaliações é o {df2_aux.iloc[0,0]} com {df2_aux.iloc[0,1]} avaliações")

O restaurante que possui a maior quantidade de avaliações é o Domino's Pizza com 59749 avaliações


## 2. Qual o nome do restaurante com a maior nota média?

In [ ]:
df2_aux = df2.loc[:,['restaurant_name','aggregate_rating','restaurant_id']].groupby(['restaurant_name','restaurant_id']).mean().reset_index().sort_values(by='aggregate_rating',ascending=False)
apenas_melhores_rest = df2_aux['aggregate_rating'] == df2_aux.iloc[0,2]
df2_desempate = df2.loc[apenas_melhores_rest,['restaurant_id','restaurant_name']].sort_values(by='restaurant_id',ascending=True)
print(f"o resturante com a maior nota média é o {df2_desempate.iloc[0,1]} com média de {df2_aux.iloc[0,2]}")

o resturante com a maior nota média é o Cafe 1730 com média de 4.9


## 3. Qual o nome do restaurante que possui o maior valor de um prato para duas pessoas?

In [ ]:
df2_aux = df2.loc[:,['restaurant_name','average_cost_for_two']].groupby('average_cost_for_two').max().reset_index().sort_values(by='average_cost_for_two',ascending=False)
print(f"O restaurante que possui o maior valor de um prato para 2 é {df2_aux.iloc[0,1]} com o valor de {df2_aux.iloc[0,0]} dólares.")

O restaurante que possui o maior valor de um prato para 2 é Corner House com o valor de 755.0 dólares.


## 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [ ]:
filtro_br = df2['cuisines'] == 'Brazilian'
df2_aux = df2.loc[filtro_br,['restaurant_name','aggregate_rating','restaurant_id']].groupby(['restaurant_name','restaurant_id']).mean().reset_index().sort_values(by='aggregate_rating',ascending=True)
filtro_melhores = df2_aux['aggregate_rating'] == df2_aux.iloc[0,2]
df2_desempate = df2_aux.loc[filtro_melhores,['restaurant_name','restaurant_id']].sort_values(by='restaurant_id',ascending=True)
print(f"O restaurante de culinária brasileira que possui a menor média de avaliação é {df2_desempate.iloc[0,0]} com nota de {df2_aux.iloc[0,2]}")

O restaurante de culinária brasileira que possui a menor média de avaliação é Loca Como tu Madre com nota de 0.0


## 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [ ]:
filtro_apenas_br = ((df2['cuisines'] == 'Brazilian') & (df2['country_code'] == 'Brazil'))
df2_br = df2.loc[filtro_apenas_br,['aggregate_rating','restaurant_name','restaurant_id']].groupby(['restaurant_name','restaurant_id']).mean().reset_index().sort_values(by='aggregate_rating',ascending=False)
df2_filtro_49 = df2_br['aggregate_rating'] == df2_br.iloc[0,2]
df2_final = df2_br.loc[df2_filtro_49,['restaurant_id','restaurant_name']].sort_values(by='restaurant_id',ascending=True)
print(f"O restaurante que fica no Brasil e possui culinária brasileira que possui a maior média de avaliações é {df2_final.iloc[0,1]} com nota {df2_br.iloc[0,2]}")

O restaurante que fica no Brasil e possui culinária brasileira que possui a maior média de avaliações é Braseiro da Gávea com nota 4.9


## 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [ ]:
df2_online_deliv = df2.loc[:,['has_online_delivery','votes']].groupby(['has_online_delivery']).sum().reset_index().sort_values(by='votes',ascending=False)
print("Não há relação com a entrega online e as avaliações registradas, como podemos ver no gráfico")
px.pie(df2_online_deliv,values='votes',names=['Não Entrega Online','Entrega Online'])

Não há relação com a entrega online e as avaliações registradas, como podemos ver no gráfico


## 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [ ]:
df2_pie = df2.loc[:,['average_cost_for_two','has_table_booking']].groupby('has_table_booking').mean().sort_values(by='average_cost_for_two',ascending=False).reset_index()
print(f"Como podemos ver pelo gráfico, o valor é mais alto de um prato para duas pessoas nos restaurantes que fazem reserva")
px.pie(df2_pie,values='average_cost_for_two',names=['Faz Reserva','Não Faz Reserva'])

Como podemos ver pelo gráfico, o valor é mais alto de um prato para duas pessoas nos restaurantes que fazem reserva


## 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [ ]:
filtro_country_american = df2['country_code'] == 'United States of America'
filtro_cuisines_japanese = df2['cuisines'] == 'Japanese'
filtro_cuisines_bbq = df2['cuisines'] == 'BBQ'
filtro_country_american.unique()
df2_filtrado = df2.loc[(filtro_country_american) & ((filtro_cuisines_japanese) | (filtro_cuisines_bbq)),['cuisines','average_cost_for_two']].groupby('cuisines').mean().reset_index()
percentual = 100*((df2_filtrado.iloc[1,1] - df2_filtrado.iloc[0,1])/df2_filtrado.iloc[0,1])
print(f"Sim. Os restaurantes de tipo culinária Japonesa apresentam um valor médio de prato para duas pessoas {np.round(percentual,decimals=2)}% maior que os de tipo BBQ")


Sim. Os restaurantes de tipo culinária Japonesa apresentam um valor médio de prato para duas pessoas 42.29% maior que os de tipo BBQ


#Visão Culinária

## 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
restaurant_by_cuisine('Italian',False)

O restaurante com a maior nota média que possui apenas o tipo de culinária Italian é Darshan com a média de 4.9


## 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
restaurant_by_cuisine('Italian',True)

O restaurante com a menor nota média que possui apenas o tipo de culinária Italian é Avenida Paulista com a média de 0.0


## 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
restaurant_by_cuisine('American',False)

O restaurante com a maior nota média que possui apenas o tipo de culinária American é Burger & Lobster com a média de 4.9


## 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
restaurant_by_cuisine('American',True)

O restaurante com a menor nota média que possui apenas o tipo de culinária American é Alston Bar & Beef com a média de 0.0


## 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
restaurant_by_cuisine('Arabian',False)

O restaurante com a maior nota média que possui apenas o tipo de culinária Arabian é Mandi@36 com a média de 4.7


## 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
restaurant_by_cuisine('Arabian',True)

O restaurante com a menor nota média que possui apenas o tipo de culinária Arabian é Raful com a média de 0.0


## 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
restaurant_by_cuisine('Japanese',False)

O restaurante com a maior nota média que possui apenas o tipo de culinária Japanese é Sushi Samba com a média de 4.9


## 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
restaurant_by_cuisine('Japanese',True)

O restaurante com a menor nota média que possui apenas o tipo de culinária Japanese é Banzai Sushi com a média de 0.0


## 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [ ]:
restaurant_by_cuisine('Home-made',False)

O restaurante com a maior nota média que possui apenas o tipo de culinária Home-made é Kanaat Lokantası com a média de 4.0


## 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [ ]:
restaurant_by_cuisine('Home-made',True)

O restaurante com a menor nota média que possui apenas o tipo de culinária Home-made é GurMekan Restaurant com a média de 3.7


## 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [ ]:
df2_aux = df2.loc[:,['cuisines','average_cost_for_two']].groupby('cuisines').max().reset_index().sort_values(by='average_cost_for_two',ascending=False)
print(f"O tipo de culinária que possui o maior valor médio de um prato para duas pessoas é a {df2_aux.iloc[0,0]} com o valor de {df2_aux.iloc[0,1]}")

O tipo de culinária que possui o maior valor médio de um prato para duas pessoas é a Continental com o valor de 755.0


## 12. Qual o tipo de culinária que possui a maior nota média?

In [ ]:
df2_aux = df2.loc[:,['cuisines','aggregate_rating']].groupby('cuisines').mean().reset_index().sort_values(by='aggregate_rating',ascending=False)
print(f"O tipo de culinária que possui a maior nota média é a {df2_aux.iloc[0,0]} com a nota média de {df2_aux.iloc[0,1]}")

O tipo de culinária que possui a maior nota média é a Others com a nota média de 4.9


## 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [ ]:
filtro_online_entregas = (df2['has_online_delivery'] == 1) & (df2['is_delivering_now'] == 1)
df2_aux = df2.loc[filtro_online_entregas,['cuisines','restaurant_id']].groupby('cuisines').nunique().reset_index().sort_values(by='restaurant_id',ascending=False)
print(f"O tipo de culinária que possui mais restaurante que fazem entregas e aceitam pedidos online é a {df2_aux.iloc[0,0]}, que possui {df2_aux.iloc[0,1]} restaurantes nesse quesito")

O tipo de culinária que possui mais restaurante que fazem entregas e aceitam pedidos online é a North Indian, que possui 317 restaurantes nesse quesito


#Mapa dos restaurantes

In [ ]:
map_ = folium.Map(zoom_starts=10)
marker_cluster = MarkerCluster().add_to(map_)
for index, row in df2.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  icon=folium.Icon(icon='cutlery', color=row['rating_color']),
                  popup=f"<b>{row['restaurant_name']}</b><br><br>"
                        f"Preço: ${round(row['average_cost_for_two'], 2)}\n"#Adicionar um espaço entre o : e a continuação do dicionário faz diferença!
                        f"Avaliação:{row['aggregate_rating']}/5.0\n"
                        f"Tipo:{row['cuisines']}"
                 ).add_to(marker_cluster)
map_

Output hidden; open in https://colab.research.google.com to view.